In [12]:
#cluster via longitude and latitude and restaurant type
from matplotlib import pyplot as plt
import matplotlib as mp
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import sklearn.manifold
from scipy import cluster
import seaborn as sns
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
import requests
import json
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
import re
%matplotlib inline

In [13]:
# response = requests.get('https://api.yelp.com/v2/search?term=food&location=San+Francisco')

In [14]:
# print(response)

In [15]:
# json_response = response.json()

In [16]:
# print(json.dumps(json_response, indent=2))

In [17]:
# df = pd.DataFrame.from_csv('Food_Establishment_Inspections.csv', index_col = None)

In [18]:
# df.head()

In [19]:
# df.shape

In [20]:
# len(df['BusinessName'].unique())

In [21]:
file = 'yelp_academic_dataset_business.json'
data = []
with open(file) as f:
    for line in f:
        data.append(json.loads(line))

In [22]:
# cities = ['Phoenix', 'Phoenix Sky Harbor Center', 'Phoenix Metro Area', 'phoenix', 'Phoenix ', 'Metro Phoenix', 'Phoenix-Ahwatukee', 'Phoenix, AZ']
# filtered_city_data = []
# for business in data:
#     if business['city'] in cities:
#         filtered_city_data.append(business)

In [24]:
# cities = []
# for business in data:
#     if business['city'] not in cities:
#         cities.append(business['city'])

In [25]:
# len(filtered_city_data)

In [41]:
filtered_category_data = []
restaurant_ids = []
for business in data:
    if 'Restaurants' in business['categories']:
        filtered_category_data.append(business)
        restaurant_ids.append(business['business_id'])

In [27]:
# # import collections
# # restaurant_count = collections.defaultdict(int)
# restaurant_count = {}

# file = 'yelp_academic_dataset_business.json'
# with open(file) as f:
#     for line in f:
#         r_line = json.loads(line)
#         restaurant_count[r_line['city']] += 1

In [28]:
import collections
restaurant_count = collections.defaultdict(int)
# restaurant_count = {}

# file = 'yelp_academic_dataset_business.json'
# with open(file) as f:
#     for line in f:
for business in filtered_category_data:
#     r_line = json.loads(line)
    restaurant_count[business['city']] += 1

In [29]:
print(dict(restaurant_count))

{'Saint-Leonard': 13, 'Ft. Mill': 1, 'Terrebonne': 3, 'Swissvale': 3, 'Indian Trail': 24, 'La Prairie': 7, 'South Queensferry': 4, 'San Tan Valley': 27, 'St-Laurent': 1, 'Glendale': 522, 'Arlington': 1, 'Dollard-Des-Ormeaux': 25, 'Karlsbad': 5, 'Lower Lawrenceville': 1, 'rankin': 1, 'Concord Mills': 1, 'Kirkland': 14, 'Whitehall': 1, 'St. Agatha': 1, 'Enterprise': 1, 'Edinburgh': 1215, 'Sainte-Ann-De-Bellevue': 1, 'Phoenix': 2922, 'Stutensee-Blankenloch': 1, 'North Scottsdale': 1, 'Mount Washington': 1, 'Pittsburgh/Waterfront': 1, 'Juniper Green': 3, 'Queen Creek': 105, 'Montreal-West': 2, 'Stowe Township': 1, 'Ettlingen': 37, 'St Jacobs': 3, 'North Las Vegas': 240, 'Tonto Basin': 1, 'Glasgow': 1, 'Paradise Valley': 26, 'Champaign': 198, 'Neuburg am Rhein': 2, 'Millvale': 1, 'N Las Vegas': 8, 'Las Vegas': 4658, 'Matthews': 136, 'Boucherville': 1, 'Edgewood': 1, 'Sainte-Thérèse': 2, 'Gila Bend': 9, 'Sedona': 1, 'Charlotte': 1886, 'Paradise': 7, 'Boisbriand': 5, 'Lawrenceville': 1, 'Tega

In [30]:
sorted(restaurant_count.items(), key=lambda x:x[1], reverse=True)

[('Las Vegas', 4658),
 ('Phoenix', 2922),
 ('Montréal', 2291),
 ('Charlotte', 1886),
 ('Pittsburgh', 1401),
 ('Edinburgh', 1215),
 ('Scottsdale', 1177),
 ('Mesa', 870),
 ('Madison', 806),
 ('Tempe', 789),
 ('Henderson', 653),
 ('Chandler', 645),
 ('Glendale', 522),
 ('Karlsruhe', 455),
 ('Gilbert', 399),
 ('Peoria', 278),
 ('North Las Vegas', 240),
 ('Champaign', 198),
 ('Waterloo', 175),
 ('Surprise', 163),
 ('Laval', 156),
 ('Kitchener', 142),
 ('Goodyear', 136),
 ('Matthews', 136),
 ('Avondale', 129),
 ('Queen Creek', 105),
 ('Fort Mill', 105),
 ('Urbana', 99),
 ('Middleton', 83),
 ('Casa Grande', 79),
 ('Concord', 71),
 ('Cave Creek', 69),
 ('Verdun', 64),
 ('Apache Junction', 58),
 ('Maricopa', 57),
 ('Sun Prairie', 55),
 ('Fountain Hills', 54),
 ('Buckeye', 53),
 ('Brossard', 51),
 ('Saint-Laurent', 50),
 ('Fitchburg', 47),
 ('Pineville', 42),
 ('Montreal', 41),
 ('Wickenburg', 40),
 ('Monona', 39),
 ('Ettlingen', 37),
 ('Anthem', 36),
 ('Sun City', 35),
 ('Litchfield Park', 33),

In [31]:
max(restaurant_count, key=restaurant_count.get)

'Las Vegas'

In [32]:
restaurant_count['Las Vegas']

4658

In [33]:
# print(dict(restaurant_count))

In [34]:
# max(restaurant_count, key=restaurant_count.get)

In [35]:
# restaurant_count['Las Vegas']

In [36]:
# len(filtered_category_data)

In [37]:
# cities

In [38]:
# file = 'yelp_academic_dataset_review.json'
# data = []
# with open(file) as f:
#     for line in f:
#         data.append(json.loads(line))

In [39]:
file = 'yelp_academic_dataset_business.json'
business_locations = {}
with open(file) as f:
    for line in f:
        r_line = json.loads(line)
        business_locations.update({r_line['business_id']:r_line['city']})

In [42]:
reviews_per_city = collections.defaultdict(int)

file = 'yelp_academic_dataset_review.json'
with open(file) as f:
    for line in f:
        r_line = json.loads(line)
#         business_locations.update({r_line['business_id']:r_line['city']})
        if r_line['business_id'] in restaurant_ids:
            reviews_per_city[business_locations[r_line['business_id']]] += 1
        
# for business in filtered_category_data:
# #     r_line = json.loads(line)
#     restaurant_count[business['city']] += 1

In [44]:
print(dict(reviews_per_city))

{'Saint-Leonard': 78, 'Ft. Mill': 41, 'Terrebonne': 16, 'Swissvale': 20, 'Indian Trail': 539, 'La Prairie': 39, 'South Queensferry': 22, 'San Tan Valley': 700, 'St-Laurent': 3, 'Glendale': 21951, 'Arlington': 7, 'Dollard-Des-Ormeaux': 206, 'Karlsbad': 10, 'Lower Lawrenceville': 171, 'rankin': 18, 'Concord Mills': 25, 'Kirkland': 74, 'Whitehall': 30, 'St. Agatha': 3, 'Enterprise': 49, 'Edinburgh': 14256, 'Sainte-Ann-De-Bellevue': 4, 'Phoenix': 188504, 'Stutensee-Blankenloch': 7, 'North Scottsdale': 124, 'Mount Washington': 11, 'Pittsburgh/Waterfront': 90, 'Juniper Green': 18, 'Queen Creek': 4145, 'Montreal-West': 8, 'Stowe Township': 10, 'Ettlingen': 156, 'St Jacobs': 19, 'North Las Vegas': 8705, 'Tonto Basin': 6, 'Glasgow': 3, 'Paradise Valley': 1237, 'Champaign': 7908, 'Neuburg am Rhein': 1, 'Millvale': 8, 'N Las Vegas': 104, 'Las Vegas': 501631, 'Matthews': 3514, 'Boucherville': 3, 'Edgewood': 4, 'Sainte-Thérèse': 8, 'Gila Bend': 218, 'Sedona': 89, 'Charlotte': 83674, 'Paradise': 76,

In [45]:
sorted(reviews_per_city.items(), key=lambda x:x[1], reverse=True)

[('Las Vegas', 501631),
 ('Phoenix', 188504),
 ('Scottsdale', 102557),
 ('Charlotte', 83674),
 ('Pittsburgh', 61849),
 ('Tempe', 55273),
 ('Montréal', 45425),
 ('Henderson', 42144),
 ('Chandler', 37843),
 ('Mesa', 35362),
 ('Madison', 35290),
 ('Gilbert', 26624),
 ('Glendale', 21951),
 ('Edinburgh', 14256),
 ('Peoria', 10564),
 ('North Las Vegas', 8705),
 ('Champaign', 7908),
 ('Surprise', 6857),
 ('Goodyear', 6452),
 ('Avondale', 4738),
 ('Queen Creek', 4145),
 ('Cave Creek', 4013),
 ('Matthews', 3514),
 ('Urbana', 3271),
 ('Waterloo', 2353),
 ('Middleton', 2353),
 ('Karlsruhe', 2350),
 ('Fort Mill', 2073),
 ('Concord', 2036),
 ('Fountain Hills', 1602),
 ('Casa Grande', 1516),
 ('Maricopa', 1419),
 ('Kitchener', 1341),
 ('Anthem', 1296),
 ('Litchfield Park', 1294),
 ('Paradise Valley', 1237),
 ('Pineville', 1224),
 ('Sun Prairie', 1216),
 ('Fitchburg', 1212),
 ('Apache Junction', 1170),
 ('Laval', 1136),
 ('Homestead', 950),
 ('Buckeye', 937),
 ('Dallas', 891),
 ('Montreal', 785),
 ('